In [1]:
import langchain
from openai import OpenAI
import json

In [2]:
student_description = """
Sunny Savita is a student of Computer Science at IIT Delhi. He is an Indian and has a 8.5 GPA. Sunny is known for his programming
skills and is an active member of the college AI club. He hopes to pursue a career in Artficial Intelligence after graduating.
"""

In [3]:
# Few shot prompt
prompt = f"""
Please extract the following information from the given text and return it as a JSON object:

Name
College
Grades
Club

This is the body of text to extract information from: {student_description}
"""

In [4]:
with open("apikey.txt") as f:
    my_key = f.read()

In [7]:
client = OpenAI(api_key=my_key)

In [8]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
        "role": "user", 
        "content": prompt
    }
  ]
)

In [16]:
response.choices[0].message.content

'{\n  "Name": "Sunny Savita",\n  "College": "IIT Delhi",\n  "Grades": 8.5,\n  "Club": "AI club"\n}'

In [18]:
output = response.choices[0].message.content
json.loads(output)

{'Name': 'Sunny Savita',
 'College': 'IIT Delhi',
 'Grades': 8.5,
 'Club': 'AI club'}

In [19]:
student_custom_function = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'college': {
                    'type': 'string',
                    'description': 'The college name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'CGPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'college club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [31]:
student_custom_function = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'college': {
                    'type': 'string',
                    'description': 'The college name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'CGPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'college club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [39]:
function_two = student_custom_function

In [33]:
response2 = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{"role": "user", "content": prompt}],
    functions = student_custom_function
)

In [34]:
response2.choices[0].message.function_call.arguments

'{}'

In [35]:
json.loads(response2.choices[0].message.function_call.arguments)

{}

In [36]:
student_description_two = """
Krish Naik is a student of computer science at IIT Mumbai. He is an indian and has a 9.5 GPA. krish is known for his 
programming skills and is an active member of the college's data science Club. He hopes to pursue a career in artificial 
intelligence after graduating.
"""

In [37]:
student_description_three = """
Sudhanshu Kumar is a student of computer science at IIT Bangalore. He is an indian and has a 9.2 GPA. krish is known for his 
programming skills and is an active member of the college's MLOps Club. He hopes to pursue a career in artificial intelligence 
after graduating.
"""

In [40]:
functions = [student_custom_function[0], function_two[0]]
student_info = [student_description, student_description_two, student_description_three]
for student in student_info:
    response =  client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': student}],
        functions = functions,
        function_call = 'auto'
    )

    response = json.loads(response.choices[0].message.function_call.arguments)
    print(response)#import csv

{'name': 'Sunny Savita', 'college': 'IIT Delhi', 'grades': 8.5, 'club': 'AI club'}
{'name': 'Krish Naik', 'college': 'IIT Mumbai', 'grades': 9.5, 'club': 'data science Club'}
{'name': 'Sudhanshu Kumar', 'college': 'IIT Bangalore', 'grades': 9.2, 'club': 'MLOps Club'}


# Advanced Function Calling

In [41]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": "When's the next flight from Delhi to Mumbai?"
    }
      ]
)

In [42]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DEL",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. MUM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

In [43]:
user_prompt = "When's the next flight from new delhi to mumbai?"

response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": user_prompt
    }
      ],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
    
)

In [44]:
response2.choices[0].message.function_call.arguments

'{"loc_origin":"DEL","loc_destination":"BOM"}'

In [45]:
from datetime import datetime, timedelta
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)

In [46]:
params=json.loads(response2.choices[0].message.function_call.arguments)

In [47]:
flight = get_flight_info(**params)

print(flight)

{"loc_origin": "DEL", "loc_destination": "BOM", "datetime": "2024-04-09 17:38:08.120434", "airline": "KLM", "flight": "KL643"}


In [48]:
response3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "user","content": user_prompt},
    {"role": "function", "name": response2.choices[0].message.function_call.name, "content": flight}
      ],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
    
)